<a href="https://colab.research.google.com/github/CPJKU/partitura_tutorial/blob/mlflow/notebooks/03_mlflow/pitch_spelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. Pitch Spelling with Partitura

Have you always been bad at spelling bee, do you find that spelling notes makes this even worse. Your time of struggling is over.... Today we going to teach a Model to learn how to *pitch* spell for you.

### Definition

Spelling a pitch relates to the system of naming notes by letters (A-G) and sharp(#) and flat (♭) signs - and sometimes double sharp and flat signs, resulting in names or 'spellings' like 'A♭', 'D#', 'F♭♭'.

Translating between frequencies in Hz and such names is non-trivial. You need to consider :

- The 'concert pitch' you are taking as a reference
- The temperament in which the piece is played
- The overall key that the music would be notated in
- Use of the correct enharmonic equivalents for accidentals (Using the correct enharmonic equivalent, Purpose of double-sharps and double-flats?)

If translating between, say, MIDI note numbers and 'spelled' names, the first two steps can be skipped.

Spelled pitch names often have an octave number appended for disambiguation - e.g. 'A♭3', 'D#5'.

### Some Examples

![Pitch Spelling Example](./assets/chord_spelling.png "Pitch Spelling figure from Automatic Pitch Spelling, E. Cambouropoulos, Published in 2001")


### Some Spelling algorithms

Partitura contains an implementation for a standard algorithm for Pitch Spelling. The algorithm in question is called ps13 created by Meredith and al.:

	The ps13 pitch spelling algorithm, D Meredith - Journal of New Music Research, 2006

Some notable algorithms and currect SOTA is PKSpell.

	PKSpell: Data-driven pitch spelling and key signature estimation
	F Foscarin, N Audebert, R Fournier-S'Niehotta, 2021


Let's first download a pitch spelling dataset.

In [146]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
	!pip install partitura
	!pip install pytorch_lightning

In [1]:
import partitura as pt
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import os
import pandas as pd
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
import warnings
warnings.filterwarnings('ignore')

In [5]:
!git clone -b note_alignments --single-branch https://github.com/CPJKU/asap-dataset.git

--2022-11-07 09:52:43--  https://github.com/CPJKU/asap-dataset/archive/refs/heads/note_alignments.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/CPJKU/asap-dataset/zip/refs/heads/note_alignments [following]
--2022-11-07 09:52:44--  https://codeload.github.com/CPJKU/asap-dataset/zip/refs/heads/note_alignments
Resolving codeload.github.com (codeload.github.com)... 140.82.121.9
Connecting to codeload.github.com (codeload.github.com)|140.82.121.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘note_alignments.zip’

note_alignments.zip     [                 <=>] 248.45M  1.19MB/s    in 2m 39s  

2022-11-07 09:55:24 (1.56 MB/s) - ‘note_alignments.zip’ saved [260516656]



In [42]:
# Selecting a subset of files from the dataset (Only Bach Files for this tutorial)
asap_files = [(os.path.join(root, file), os.path.join(os.path.dirname(root), os.path.basename(root).split("_")[0]+".mid"), os.path.join(os.path.dirname(root), "xml_score.musicxml"), os.path.join(root, os.path.splitext(file)[0]+".match")) for root, dirs, files in os.walk("./content/asap-dataset-note_alignments/Bach") for file in files if file.endswith("note_alignment.tsv")]

In [46]:
# Spliting Bach files into train and test subsets.
asap_train = [t for t in asap_files if "Italian_concerto" not in t[0]]
asap_test = [t for t in asap_files if "Italian_concerto" in t[0]]

As we saw on the previous section the Match file is a very convenient format to represent an alignment between a score and a performance. For this task we are going to predict the pitch spelling from performed MIDI files and evaluate the results using the Note aligned scores.

Let's see how to produce a Match file using alignment information, a midi file and a score from the same file, using partitura.

In [47]:
def produce_match(alignment_fn, mfn, sfn, match_name):
	"""
	Produce and Save a Match file from alignment, performance, and score.

	Parameters
	----------
	alignment_fn : str
		Alignment ".tsv" file path
	mfn : str
		Performance Midi File Path
	sfn : str
		Score musicxml File Path
	match_name : str
		Path and Save Name.
	"""
	data = pd.read_csv(alignment_fn, sep="\t")

	alignment = list()
	for x in data[["xml_id", "midi_id"]].to_numpy():
		if x[1] == "deletion":
			dd = dict(label="deletion", score_id=x[0])
		elif x[0] == "insertion":
			dd = dict(label="insertion", performance_id=str(x[1]))
		else:
			dd = dict(label="match", score_id=x[0], performance_id=str(x[1]))
		alignment.append(dd)
	ppart = pt.load_performance_midi(mfn)
	# This may cause re-indexing.
	spart = pt.score.merge_parts(pt.load_musicxml(sfn))
	spart = pt.score.unfold_part_maximal(spart, ignore_leaps=False)
	pt.save_match(alignment, ppart, spart, match_name)

In [48]:
for alignment_fn, mfn, sfn, match_name in asap_files:
	produce_match(alignment_fn, mfn, sfn, match_name)

/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/io/importmidi.py:151: UserWarning: change of Tempo to mpq = 500000  and resulting seconds per tick = 0.0013020833333333333at time: 0.0
  warnings.warn(
/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/directions.py:533: UserWarning: error parsing "(   )" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/utils/generic.py:238: UserWarning: reference not found in
                            o_map: -- Tuplet start=n109 end=n116 start=None end=None, substituting None

  warnings.warn(
/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/utils/generic.py:238: UserWarning: reference not found in
                            o_map: -- Tuplet start=n798 end=n805 start=None end=None, substituting None

  warnin

In [49]:
_, _, score_files, match_files = zip(*asap_files)
asap_train = [t for t in zip(score_files, match_files) if "Italian_concerto" not in t[0]]
asap_test = [t for t in zip(score_files, match_files) if "Italian_concerto" in t[0]]

To train a pitch spelling model we will need some global description of pitches to perform tokenization.

| **Pitch Class** | **Tonal Pitch Class** |
|-----------------|-------------------|
| 11              | B♮,  C♭,  A𝄪     |
| 10              | B♭,  A♯,  C♭      |
| 9               | A♮,  G♭,  B𝄫       |
| 8               | A♭,  G♯           |
| 7               | G♮,  F♭,  A𝄫       |
| 6               | F♯,  G♭,  E𝄪       |
| 5               | F♮,  E♯,  G𝄫       |
| 4               | E♮,  F♭,  D𝄪     |
| 3               | D♯,  E♭,  F𝄫       |
| 2               | D♮,  C𝄪,  E𝄫      |
| 1               | C♯,  D♭,  B𝄪     |
| 0               | C♮,  B♯,  D𝄫       |

In [70]:
PAD = "<PAD>"

PITCHES = {
	0: ["C", "B#", "D--"],
	1: ["C#", "B##", "D-"],
	2: ["D", "C##", "E--"],
	3: ["D#", "E-", "F--"],
	4: ["E", "D##", "F-"],
	5: ["F", "E#", "G--"],
	6: ["F#", "E##", "G-"],
	7: ["G", "F##", "A--"],
	8: ["G#", "A-"],
	9: ["A", "G##", "B--"],
	10: ["A#", "B-", "C--"],
	11: ["B", "A##", "C-"],
}

accepted_pitches = [ii for i in PITCHES.values() for ii in i]
pitch_to_ix = {p: accepted_pitches.index(p) for p in accepted_pitches}
# add PADDING TAD
pitch_to_ix[PAD] = len(accepted_pitches)

def tokenize_pitch_spelling(ps_note):
	# step = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6}[]
	alter = {0:"", 1:"#", 2:"##", -1:"-", -2:"--"}[ps_note["alter"].item()]
	return pitch_to_ix[ps_note["step"].item()+alter]

def extract_features(perf_note):
	features = np.zeros((14,))
	# One hot of Pitch Class for first 12 entries
	features[int(perf_note["pitch"].item()%12)] = 1
	# pitch as float
	features[12] = perf_note["pitch"].item()/127
	# duration normalized per minute
	features[13] = perf_note["duration_sec"].item() / 60
	return features

def create_data(files):
	data, labels = list(), list()
	for score_file, match_file in files:
		performance, alignment = pt.load_match(match_file)
		score = pt.load_score(score_file)
		spart = pt.score.merge_parts(score)
		spart = pt.score.unfold_part_maximal(spart, ignore_leaps=False)
		matched_notes = [alignment[idx] for idx, d in enumerate(alignment) if d["label"] == "match"]
		pna = performance.note_array()
		sna = spart.note_array(include_pitch_spelling=True)
		X, y = np.zeros((len(matched_notes), 14), dtype=float), np.zeros((len(matched_notes), ), dtype=int)
		for idx, match_note in enumerate(matched_notes):
			X[idx] = extract_features(pna[np.where(pna["id"] == str(match_note["performance_id"]))])
			y[idx] = tokenize_pitch_spelling(sna[np.where(sna["id"] == match_note["score_id"])][["step", "alter", "octave"]])
		data.append(X)
		labels.append(y)
	return data, labels

In [71]:
X_train, y_train = create_data(asap_train)
X_test, y_test = create_data(asap_test)

/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/utils/generic.py:238: UserWarning: reference not found in
                            o_map: -- Tuplet start=n1 end=n8 start=None end=None, substituting None

  warnings.warn(
/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/utils/generic.py:238: UserWarning: reference not found in
                            o_map: -- Tuplet start=n51 end=n58 start=None end=None, substituting None

  warnings.warn(
/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/utils/generic.py:238: UserWarning: reference not found in
                            o_map: --1204 Slur start=n1086 end=n1087 start=None end=TimePoint t=1204 quarter=8, substituting None

  warnings.warn(
/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/utils/generic.py:238: UserWarning: reference not found in
                            o_map: --1

### Model

In [148]:
class PKSpell(nn.Module):
	"""Models that decouples key signature estimation from pitch spelling by adding a second RNN.
	This model reached state of the art performances for pitch spelling.
	"""

	def __init__(
		self,
		input_dim=14,
		hidden_dim=100,
		pitch_to_ix=pitch_to_ix,
		hidden_dim2=24,
		rnn_depth=1,
		dropout=0.1,
		bidirectional=True
	):
		super(PKSpell, self).__init__()
		self.dropout = nn.Dropout(dropout)
		self.n_out_pitch = len(pitch_to_ix)
		self.hidden_dim = hidden_dim
		self.hidden_dim2 = hidden_dim2

		# RNN layer.
		self.rnn = nn.LSTM(
			input_size=input_dim,
			hidden_size=hidden_dim // 2 if bidirectional else hidden_dim,
			bidirectional=bidirectional,
			num_layers=rnn_depth,
		)
		# Output layers.
		self.top_layer_pitch = nn.Linear(hidden_dim, self.n_out_pitch)
		# Loss function that we will use during training.
		self.loss_pitch = nn.CrossEntropyLoss()

	def compute_outputs(self, sentences, sentences_len):
		rnn_out, _ = self.rnn(sentences)
		rnn_out = self.dropout(rnn_out)
		out_pitch = self.top_layer_pitch(rnn_out)
		return out_pitch

	def forward(self, sentences, pitches, sentences_len):
		# First computes the predictions, and then the loss function.

		# Compute the outputs. The shape is (max_len, n_sentences, n_labels).
		scores_pitch = self.compute_outputs(sentences, sentences_len)

		# Flatten the outputs and the gold-standard labels, to compute the loss.
		# The input to this loss needs to be one 2-dimensional and one 1-dimensional tensor.
		scores_pitch = scores_pitch.view(-1, self.n_out_pitch)
		loss = self.loss_pitch(scores_pitch, pitches)
		acc = (scores_pitch.argmax(dim=-1) == pitches).float().mean()
		return loss, acc

	def predict(self, ppart):
		# Compute the outputs from the linear units.
		pna = ppart.note_array()
		features = np.zeros((len(pna),14))
		features[np.arange(len(pna)), np.remainder(pna["pitch"], 12)] = 1
		features[:, 12] = pna["pitch"]/127
		features[:, 13] = pna["duration_sec"] / 60
		scores_pitch = self.compute_outputs(torch.tensor([features]).float(), [len(features)])
		# Select the top-scoring labels.
		predicted_pitch = scores_pitch.argmax(dim=2).squeeze()
		spelling_array = [(accepted_pitches[pp][0], {"":0, "#":1, "##":2, "-":-1, "--":-2}[accepted_pitches[pp][1:]], int(pna[i]["pitch"].item()/12)-1)for i, pp in enumerate(predicted_pitch)]
		out = np.array(spelling_array, dtype=[('step', '<U1'), ('alter', '<i8'), ('octave', '<i8')])
		return out

class PSDataset(Dataset):
	def __init__(self, x, y):
		super(PSDataset, self).__init__()
		self.x = x
		self.y = y
	def __getitem__(self, idx):
		return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])
	def __len__(self):
		return len(self.x)

def collate_ps(data):
	def merge(sequences):
		lengths = [len(seq) for seq in sequences]
		padded_seqs = torch.zeros(len(sequences), max(lengths)).long()
		for i, seq in enumerate(sequences):
			end = lengths[i]
			padded_seqs[i, :end] = seq[:end]
		return sequences, lengths

	# sort a list by sequence length (descending order) to use pack_padded_sequence
	data.sort(key=lambda x: len(x[0]), reverse=True)

	# seperate source and target sequences
	src_seqs, trg_seqs = zip(*data)

	# merge sequences (from tuple of 1D tensor to 2D tensor)
	# src_seqs, src_lengths = merge(src_seqs)
	# trg_seqs, trg_lengths = merge(trg_seqs)
	src_lengths = [len(seq) for seq in src_seqs]

	return src_seqs[0].float(), src_lengths, trg_seqs[0]

class PKSpellPL(pl.LightningModule):
	def __init__(self):
		super(PKSpellPL, self).__init__()
		self.module = PKSpell()
	def training_step(self, batch, batch_idx):
		src_seqs, src_lengths, trg_seqs = batch
		loss, acc = self.module(src_seqs, trg_seqs, src_lengths)
		self.log("train_loss", loss.item(), on_epoch=True, on_step=True, prog_bar=True)
		self.log("train_acc", acc.item(), on_epoch=True, on_step=True, prog_bar=True)
		return loss
	def validation_step(self, batch, batch_idx):
		src_seqs, src_lengths, trg_seqs = batch
		loss, acc = self.module(src_seqs, trg_seqs, src_lengths)
		self.log("val_loss", loss.item(), on_epoch=True, prog_bar=True)
		self.log("val_acc", acc.item(), on_epoch=True, on_step=True, prog_bar=True)
		return loss

	def configure_optimizers(self):
		optimizer = torch.optim.Adam(self.parameters(), lr=0.001, weight_decay=5e-4)
		return {
			"optimizer": optimizer,
		}

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name   | Type    | Params
-----------------------------------
0 | module | PKSpell | 30.0 K
-----------------------------------
30.0 K    Trainable params
0         Non-trainable params
30.0 K    Total params
0.120     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


### Train the PKSpell model

In [142]:
model = PKSpellPL()
train_dataloader = DataLoader(PSDataset(X_train, y_train), collate_fn=collate_ps, batch_size=1, num_workers=4)
val_dataloader = DataLoader(PSDataset(X_test, y_test), collate_fn=collate_ps, batch_size=1, num_workers=4)
trainer = pl.Trainer(max_epochs=10)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [143]:
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)


  | Name   | Type    | Params
-----------------------------------
0 | module | PKSpell | 30.0 K
-----------------------------------
30.0 K    Trainable params
0         Non-trainable params
30.0 K    Total params
0.120     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [144]:
score_file, match_file = asap_test[0]
score = pt.score.unfold_part_maximal(pt.score.merge_parts(pt.load_score(score_file)), ignore_leaps=False)
performance, alignment = pt.load_match(match_file)
baseline_spelling = pt.musicanalysis.estimate_spelling(performance)
with torch.no_grad():
	pk_spelling = model.module.predict(performance)
sna = score.note_array(include_pitch_spelling=True)
matched_notes = [alignment[idx] for idx, d in enumerate(alignment) if d["label"] == "match"]
score_idxs = np.array([np.where(sna["id"] == match_note["score_id"])[0].item() for match_note in matched_notes])
true_spelling = sna[score_idxs][["step", "alter", "octave"]]

/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/utils/generic.py:238: UserWarning: reference not found in
                            o_map: --552 Slur start=n233 end=n234 start=None end=TimePoint t=552 quarter=16, substituting None

  warnings.warn(
/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/utils/generic.py:238: UserWarning: reference not found in
                            o_map: --188 Slur start=n42 end=n47 start=None end=TimePoint t=188 quarter=16, substituting None

  warnings.warn(
/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/utils/generic.py:238: UserWarning: reference not found in
                            o_map: --784 Slur start=n348 end=n354 start=None end=TimePoint t=784 quarter=16, substituting None

  warnings.warn(
/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/utils/generic.py:238: UserWarning: reference not 

In [145]:
pk_spelling

array([('F', 0, 5), ('D', 0, 5), ('D', 0, 4), ..., ('D', 0, 5),
       ('D', 0, 4), ('F', 0, 4)],
      dtype=[('step', '<U1'), ('alter', '<i8'), ('octave', '<i8')])

In [140]:
baseline_spelling

array([('F', 0, 4), ('D', 0, 4), ('D', 0, 3), ..., ('D', 0, 4),
       ('D', 0, 3), ('F', 0, 3)],
      dtype=[('step', '<U1'), ('alter', '<i8'), ('octave', '<i8')])